In [ ]:
# 包引用

# 13 部署软件

如果软件能在一个系统上工作，那么也能在其他相同的系统上运行。虚拟化或其他形式的工具能够做到这一点。
**虚拟化是将软件与运行的系统隔离开，主要有两种形式：**
1.虚拟机，即在一个操作系统中安装一个新操作系统。
2.虚拟环境，即在一个操作系统中创建一个新的、干净的空间来运行代码，这个空间与操作系统和环境的其他部分安全隔离。

大型虚拟化部署一般用于超级计算机和云中。
超级计算机和较小的集群中通常有与用户连接的登录机器，用来启动作业、编译代码，以及执行其他任务等。执行软件的节点通常都运行相同的操作系统和环境。
在云中，可能存在多种虚拟化来满足不同的应用程序需求。所有这些情况中，都有确定的虚拟环境。
**大型部署有一个共同点，即不允许用户直接接触软件或环境。**

## 13.1 部署软件本身

部署的第一阶段是打包软件，用来创建可分发给大众的文件。
一旦用户拥有打包文件，就可以用几个特殊命令运行代码，以在自己的系统上安装软件。

**部署方式：**
**软件包管理器：** 负责在用户计算机上安装其他软件的特殊软件。大多数操作系统现在至少预装了一个软件包管理器。
**源代码编译：** 向用户提供源代码的下载链接，让他们按照说明、手动安装代码。

**包管理的分类：**
1．源码安装：包管理器下载源代码并将其安装到用户的机器上，适用于动态语言，安装速度快，在运行软件时才会出现错误。但不适用于编译语言，因为这需要用户在系统上有一个编译器，仅Linux原生支持。
2．二进制安装：同时适用于动态和编译语言，开发人员将代码编译为对应架构组合的二进制形式。架
构由字长（32位、64位）和操作系统（Linux、Mac OS X、Windows）决定。然后将编译结果添加到ZIP文件并提供给用户，由用户解压缩到适当的位置。速度快且不易出错。
3．虚拟化安装：打包虚拟环境并发送给用户，类似于二进制包，开发人员预先创建一个只为用户工作的代码版本。但需要用户花费一些额外的步骤来创建软件的运行环境。

### 13.1.1 pip

#### pip使用

Python 打包管理机构（PyPA，Python Packaging Authority）推荐采用pip来安装Python代码。

pip之前：distutils模块。
对大多数科学软件来说，distutils有缺陷，无法很好地处理来自其他语言的编译代码，只能初步满足科学家的需求。

pip是一个命令行，它基于源代码的包管理器，用于从Python包索引（Python Package Index，PyPI）中查找和下载软件。
**pip帮助文档的部分摘录：**
```
$ pip –h
Usage:
  pip <command> [options]
Commands:
  install                    Install packages.
  uninstall                  Uninstall packages.
  list                       List installed packages.
  show                       Show information about installed packages.
  search                     Search PyPI for packages.
  help                       Show help for commands.
General Options:
  -h, --help                 Show help.
  -V, --version              Show version and exit.
  -q, --quiet                Give less output.
```

安装命令：<code>pip install packagename</code>

#### pip打包方法

此处需要使用setuotools来打包。

在目录结构的顶层创建一个名为setup.py的文件，该文件应该在要安装的模块之外。
setup.py文件与源代码目录同级。

setup.py唯一的目的是导入并调用带有适当参数的setup()函数，充当主函数，提供一个命令行界面，用于在本地安装软件以及制作软件包并上传到PyPI中。
setup()函数接收一些关键字参数，用来描述如何在文件系统上安排包源码以及应如何在用户的文件系统上安装软件包。

In [ ]:
try:
    from setuptools import setup  # 先尝试使用setuptools
    have_setuptools = True
except ImportError:
    from distutils.core import setup  # 如果不可用则使用distutils
    have_setuptools = False
setup_kwargs = {  # 在调用setup()之前创建包元数据
    'name': 'compphys',
    'version': '0.1',
    'description': 'Effective Computation in Physics',
    'author': 'Anthony Scopatz and Kathryn D. Huff',
    'author_email': 'koolkatz@gmail.com',
    'url': 'https://www.oreilly.com/',
    'classifiers': [
        'License :: OSI Approved',
        'Intended Audience :: Developers',
        'Programming Language :: Python :: 3',
        ],
    'zip_safe': False,
    'packages': ['compphys', 'compphys.more'],
    'package_dir': {
        'compphys': 'compphys',
        'compphys.more': 'compphys/more',
        },
    'data_files': [('compphys/raw', ['*.txt'])],
    }
if __name__ == '__main__':
    setup(**setup_kwargs)  # 将setup()视为主函数来调用

使用setup.py的方法：

```
command: python setup.py -h
Common commands: (see '--help-commands' for more)
  setup.py build     will build the package underneath 'build/'
  setup.py install   will install the package
Global options:
  --verbose (-v)              run verbosely (default)
  --quiet (-q)                run quietly (turns verbosity off)
  --dry-run (-n)              don't actually do anything
  --help (-h)                 show detailed help message
usage: setup.py [global_opts] cmd1 [cmd1_opts] [cmd2 [cmd2_opts] ...]
   or: setup.py --help [cmd1 cmd2 ...]
   or: setup.py --help-commands
   or: setup.py cmd –help
```

build命令将软件构建到build/目录中，如果目录不存在则会创建该目录。
install命令将build目录的内容安装到系统（当前登录的计算机）上。
使用--user标志让软件安装到用户的主目录。
如果未运行build命令，install命令将自动运行build命令。
安装到用户空间：<code>python setup.py install -–user</code>

sdist命令：使用安装脚本创建纯源代码包。构建软件包并将其放入一个ZIP文件中，使用pip解压缩和安装。
<code>python setup.py sdist</code>
PyPI是发布Python源代码最简单的方式。这是因为PyPI可与setuptools和distutils协同工作，且PyPI是免费的。
在上传软件包之前，必须先在PyPI服务器使用下面的register命令注册，这样可以确保软件包不会重名。
PyPI上有一个账户，并且在setup.py中提供有关该软件包当前状态（如版本号）的一些元数据。
注册软件包之后，就可以使用upload命令将其复制到PyPI。
操作命令：
注册：<code>python setup.py register</code>
上传：<code>python setup.py sdist upload</code>

### 13.1.2 Conda

Conda是一个跨平台二进制包管理器，旨在解决部署科学软件出现的问题。

Conda特性：
1．通常能够无缝处理多语言和非Python代码项目。
2．可以在任何操作系统上运行，特别是Linux、Mac OS、Windows。
3．默认情况下软件运行在用户空间中，不尝试安装到系统中。
科学计算领域同时需要这三个特性。

Miniconda：一个软件包，用来将Conda及其依赖项（包括Python）安装到用户指定的目录中。
默认安装位置为~/miniconda。Conda在这里搜索和安装软件包，其中包括Conda本身的更新。

Conda命令：

```
command: conda -h
usage: conda [-h] [-V] command ...
conda is a tool for managing environments and packages.
positional arguments:
  command
    help             Displays a list of available conda commands and their
 help
                     strings.
    list             List linked packages in a conda environment.
    search           Search for packages and display their information.
    create           Create a new conda environment from a list of specified packages.
    install          Install a list of packages into a specified conda
                     environment.
    update           Update conda packages.
    remove           Remove a list of packages from a specified conda
                     environment.
    clean            Remove unused packages and caches
    build            Build a package from a (conda) recipe. (ADVANCED)
```

安装命令：<code>conda install numpy</code>
从第一个频道（channel）中获取与用户平台匹配的numpy软件包。频道是指向频道文件的URL，其中含有描述频道上可用的包及其下载地址的元数据。

添加频道：<code>conda config --add channels https://conda.binstar.org/foo </code>

Conda自带一个默认频道，其中包含许多核心包。
Binstar列出了其他开发者提供的频道。

Conda不需要像pip一样设置脚本，而是查找含有某种结构的目录，即recipe。
recipe中包含的文件：
1.build.sh（用于在Linux、Mac和其他POSIX系统上构建软件的bash构建脚本）。
2.bld.bat（用于在Windows上构建的批处理脚本）。
3.meta.yaml（YAML格式的软件包元数据）。
4.run_test.py（可选文件，用于运行测试）。
5.可选文件，源代码补丁。
6.其他可选文件。
各种信息参考Conda构建文档。
编写完Conda recipe之后，使用以下命令构建软件包：
<code>conda build packagename</code>
安装Binstar：<code>conda install binstar</code>
上传到Binstar中（需要账户）:

```
binstar login
conda build packagename
```

### 13.1.3 虚拟机

虚拟机（VM）是在另一台计算机上作为进程运行的模拟计算机。
模拟计算机称为客户机，运行模拟机的计算机称为主机。
虚拟机可以嵌套，但层数不建议太多。
虚拟机并不是完全模拟的。准确来说，VM是管理程序。

对于虚拟机，管理程序将来自客户内核的时间和空间请求转换为对主机内核的相应请求，通常挂到主机的操作系统中。
客户端在主机的处理器上获得实际的计算时间，但主机的其他部分并不了解这一情况。
设置虚拟机需要指定将创建的计算机的所有属性，包括处理器数量、硬盘大小、内存大小等。

迁移到VM能够提高可靠性和可重复性。
虚拟机快照还可作为文件存储。
虚拟机对大规模部署非常重要，即使是小范围的工作，部署一个有效的虚拟机对软件也很有帮助。将VM上传到互联网，用户还可以下载并直接尝试编写的代码，而不用去了解安装过程。

### 13.1.4 Docker

容器可以认为是操作系统级别的虚拟化。
容器没有单独的管理程序，操作系统本身为客户机提供了一个接口，供客户请求和访问资源，并安全地屏蔽操作系统的其他部分。
比虚拟机更快速。

#### 使用Docker

***此处为Ubuntu中的使用说明***

使用Docker等同于使用Docker Hub，Docker Hub是Docker的在线托管服务。
安装命令：<code>sudo apt- get install -y docker docker.io</code>
运行"Hello, World!"容器：<code>sudo docker run ubuntu:22.04 echo "Hello, World!"</code>
使用run命令执行docker，从Docker Hub下载Ubuntu 22.04映像，由ubuntu:22.04指定。剩下的参数是希望从容器内部运行的bash命令。
可以指定镜像获取途径。

```
 sudo docker run ubuntu:22.04 echo "Hello, World!"  # 注意，需要root权限才能运行Docker。

[sudo] password for scopatz:  # 输入密码。
Unable to find image 'ubuntu:22.04' locally  # Docker会智能地隐藏容器以供将来使用
Pulling repository ubuntu  # 自动下载镜像
c4ff7513909d: Download complete
511136ea3c5a: Download complete
1c9383292a8f: Download complete
9942dd43ff21: Download complete
d92c3c92fa73: Download complete
0ea0d582fd90: Download complete
cc58e55aa5a5: Download complete
Hello, World!  # echo命令的输出
```
下载的映像会缓存起来以供将来使用，重新运行相同的命令无需再次下载映像。第二次就会只看到echo的输出：

```
 sudo docker run ubuntu:14.04 echo "Hello, World!"
Hello, World!
```

打印出本地系统上所有Docker镜像的列表：<code>sudo docker images</code>
pull命令提前下载：<code>sudo docker pull wordpress:latest</code>
还可以使用rmi命令：<code>sudo docker rmi wordpress</code>

#### 向Ubuntu容器添加内容

**第一步：**以交互模式启动Ubuntu容器，使用run命令和-t选项创建一个终端，-i选项表示交互模式。同时运行bash以在容器内部操作。
<code>sudo docker run -t -i ubuntu:14.04 /bin/bash</code>
**第二步：**在容器的shell中，Docker会自动记录所有操作。下面来安装Ubuntu 包管理器，并安装NumPy，然后离开。

```
 sudo docker run -t -i ubuntu:22.04 /bin/bash
root@ae37c22b3c49:/# apt-get update（1）
...
root@ae37c22b3c49:/# apt-get install -y python-numpy
...
root@ae37c22b3c49:/# exit
```

回到宿主机，为了保存工作，必须提交改动。这会创建一个基于原始映像的新映像，其中含有所有改动以及提供的元数据。
docker commit命令使用在容器中看到的标识符，通过-m选项设置消息字符串，通过-a选项设置作者名和新映像的代码库名称（此处为ubuntu- numpy）。

```
 sudo docker commit -m "with numpy" -a "<name>" ae37c22b3c49 <user>/ubunt
u-numpy
73188d24344022203bee5ef5d6cb31ccaa8b5f38085ae69fcf9502828220f81d
```

新容器现在显示在映像列表中，可供将来使用。

由于NumPy是预先加载的，因此运行docker with <user>/ubuntu-numpy能够节省时间。
用户还可以使用Dockerfile构建同一个容器。
设置Docker file文件需要花费一点精力，但之后可以重用这些设置。
对于大多数常规的Docker任务而言，交互式shell已经足够了。

#### Docker发布

使用push命令发布。先登录Docker Hub，然后使用命令：
<code>sudo docker push scopatz/ubuntu-numpy</code>
输出：

```
The push refers to a repository [scopatz/ubuntu-numpy] (len: 1)
Sending image list
Please login prior to push:
Username: scopatz
Password:
Email: scopatz@gmail.com
Login Succeeded
The push refers to a repository [scopatz/ubuntu-numpy] (len: 1)
Sending image list
Pushing repository scopatz/ubuntu-numpy (1 tags)
511136ea3c5a: Image already pushed, skipping
1c9383292a8f: Image already pushed, skipping
9942dd43ff21: Image already pushed, skipping
d92c3c92fa73: Image already pushed, skipping
0ea0d582fd90: Image already pushed, skipping
cc58e55aa5a5: Image already pushed, skipping
c4ff7513909d: Image already pushed, skipping
73188d243440: Image successfully pushed
Pushing tag for rev [73188d243440] on {https://cdn-registry-1.docker.io/v1 /repositories/scopatz/ubuntu-numpy/tags/latest}
```

## 部署到云端

云是为本地用户或机器提供广泛服务的一组远程计算机，可以看作互联网的另一种表达形式。

**云的三个部署和交互策略**
1.软件即服务（SaaS）
2.平台即服务（PaaS）
3.基础设施即服务（IaaS）

云是部署技术（包括部署人员、规模、冗余级别）的一个演化。
云技术将服务委派给更大、更稳定，且网络连接更好的第三方（例如Google、Amazon、Rackspace）来解决Slashdot效应。
云技术能根据需要向上或向下扩展服务。
云服务提供商可以根据需要轻松地配置新计算机、带宽、存储空间。
这些资源在使用完后还可以轻松删除。

#### 云技术栈

从云技术栈的底部开始，基础架构即服务用于租用大型计算机系统的部分或全部物理部件。
用户没有获得操作系统，但可以根据需求配置临时使用的系统类型。
基于IaaS最大限度地提高灵活性，最大限度地降低风险。

云技术栈的中间是平台即服务的供应商。
基于PaaS模型编写应用程序并部署。
PaaS提供了通用的方法来简化应用程序的设置和编写。
PaaS应用程序通常在IaaS机器上运行。

云技术栈的顶部是面向用户的软件即服务。
用户的交互方式，各种面向不同用户的网站基本上都使用这些工具。

云服务提供商根据客户需求提供IaaS、PaaS、SaaS。
出售虚拟机处理器上的计算时间的业务，通常最终还会添加一个Web界面来管理这些虚拟机。

云端的力量来自于专业化，即将部分业务细节的管理工作外包给专业人员。
如何部署到特定平台取决于业务需求和云供应商。
每个公司都有自己的指南和文档。先要明确需求，然后在网上调研，最后再选择供应商。

在物理科学中，当目前可用资源无法满足需求时，一般会使用云技术，而不是购买和设置昂贵的新设备，只有在需要时才租用相关资源。
租价远低于自行购置的价格。

云以高吞吐量计算模型为中心。

## 13.3 在超算上部署

**在超级计算环境中的部署的特征：**
1.跨节点的统一处理器和内存。
2.并行文件系统。
3.调度程序。

超级计算机中具有相同的处理器和相同的可用内存，意味着为机器的一部分编写的代码同样可在其他部分上执行。
用户无法直接访问超级计算机的计算节点，必须先登录到登录节点，然后提交要由调度程序执行的作业。
通常超级计算环境要求编译特定版本的代码来运行。
每一台超算都有自己的特点，为一台超级计算机进行的设置不一定适用于其他超算。
从一代机器迁移到下一代机器时，设置通常会失效。

**超级计算机的并行文件系统**
许多人和许多进程会尝试同时访问文件，而从用户角度来看，很像笔记本电脑、台式机或闪存驱动器中正常的文件系统，但性能差异很大。
并行文件系统上，访问每个文件都会产生一定的开销。由于每个文件都必须单独检查，因此大量文件的访问开销将难以忍受，因此必须限制访问的文件数量。
文件元数据在这些系统上慢得多。

通过调度程序才能在超级计算机上实际运行代码。
**负责跟踪：** 每个用户分配了多少时间，使用了多少，还有确定运行哪些作业，哪些作业在队列中等待下一个作业完成。
几乎所有的调度器都有时间限制。如果作业超过时间限制，则调度程序会突然终止作业。该时间限制通常为三天至一周。
较小和较短的作业在队列中的通过速度比较大和较长的作业更快。

<code></code>